In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
funda_data = pd.read_csv('Fundamental_data.csv')
funda_data.columns
funda_data = funda_data.drop(columns=['public_date',"pcf",'staff_sale','PEG_trailing','gvkey','permno','adate'])
funda_data = funda_data.drop_duplicates()

In [21]:
def merge_left(funda_stock,name,freq):
    price = pd.read_csv('../stock_price/'+freq+'/'+name+'.csv')
    price['Date'] = price.Date.apply(lambda x: dt.datetime.strptime(x,'%Y-%m-%d'))
    close = price.loc[:,['Date']]
    df = pd.merge(funda_stock,close,how='right',left_on=['qdate'],right_on=['Date'])
    df.fillna(method='ffill',inplace=True)
    df.set_index('Date',inplace=True)
    df.drop(columns=['qdate','TICKER'],inplace=True)
    df.dropna(inplace=True,how='all')
    
    df.to_csv('FundaIndicators/'+freq+'/'+name+'.csv')

In [ ]:
for name in pd.unique(funda_data.TICKER).tolist():
    funda_stock = funda_data.loc[funda_data.TICKER==name,:]
    funda_stock['qdate'] = funda_data.qdate.apply(lambda x:dt.datetime.strptime(x,'%m/%d/%Y'))
    if name == 'FB':
        name = 'META'
    elif name == 'BRK':
        name = 'BRK-B'
    elif name == 'GOOGL':
        name = 'GOOG'
    merge_left(funda_stock,name,'Daily')
    merge_left(funda_stock,name,'Weekly')
    merge_left(funda_stock,name,'Monthly')
    funda_stock = funda_data.loc[funda_data['TICKER'] == name,:]